### 1. **Monte um passo a passo para o algoritmo RF:**
O Random Forest possui três etapas:

Passo 1. Bootstrap + Feature Selection: o primeiro passo é similar ao bootstrap do Bagging (ou seja, é feita uma amostragem com reposição a partir da qual são criados subconjuntos de um DataFrame inicial), porém com seleção de colunas. A seleção do número de colunas é feita conforme um cálculo, que varia de acordo com a tarefa (regressão ou classificação).

Passo 2. Base learners: assim como no Bagging, a partir de cada subconjunto criado é treinado um modelo em paralelo. Porém, aqui no Random Forest, o modelo treinado é o modelo de árvore de decisão.

Passo 3. O resultado final é calculado através de um "soft voting" ou "sabedoria das multidões". Nesse processo, a classe mais frequente de resultados é a classe final.

### 2. **Explique com suas palavras o Random forest:**
Random Forest é um método de aprendizado por conjunto construído em cima do Bagging, uma espécie de extensão. Foi criado em 1996 por Leo Breiman e Adele Cutler.

###3. Qual a diferença entre Bagging e Random Forest?
A principal diferença entre o Baggine e o RF é a feature selection durante o processo de reamostragem. Essa aleatoriedade de funcionalidades gera um subconjunto aleatório de funcionalidades, o que gera baixa correlação entre as árvores de decisão. Enquanto as árvores de decisão consideram todas as possíveis divisões de funcionalidades, as florestas aleatórias selecionam apenas um subconjunto dessas funcionalidades.

### 4. (Opcional) Implementar em python o Random Forest

In [ ]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Optional, Literal, List, Tuple
from sklearn.base import clone
from sklearn.metrics import accuracy_score, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from joblib import Parallel, delayed # Para paralelização

TaskType = Literal["classification", "regression"]
VoteType = Literal["hard", "soft"]

In [ ]:
@dataclass
class RandomForestManual:
    base_estimator: object
    n_estimators: int = 200
    task: TaskType = "classification"
    voting: VoteType = "soft"  # Adicionado: Suporte a Soft Voting
    m_features: Optional[int] = None
    bootstrap: bool = True
    n_jobs: int = -1           # Adicionado: Controle de paralelismo (-1 usa todos os cores)
    random_state: int = 42

    models_: Optional[List[object]] = None
    oob_indices_: Optional[List[np.ndarray]] = None
    feature_subsets_: Optional[List[List[str]]] = None
    oob_score_: Optional[float] = None
    classes_: Optional[np.ndarray] = None

    def _rng(self):
        return np.random.default_rng(self.random_state)

    def _bootstrap_indices(self, n: int, rng: np.random.Generator):
        if not self.bootstrap:
            return np.arange(n), np.array([], dtype=int)
        inbag = rng.integers(0, n, size=n)
        oob = np.setdiff1d(np.arange(n), np.unique(inbag))
        return inbag, oob

    def _choose_features(self, X: pd.DataFrame, rng: np.random.Generator):
        p = X.shape[1]
        if self.m_features is None:
            m = int(np.sqrt(p)) if self.task == "classification" else max(1, int(p / 3))
        else:
            m = self.m_features
        m = max(1, min(m, p))
        return list(rng.choice(X.columns, size=m, replace=False))

    def _train_single_tree(self, X, y, seed):
      rng_b = np.random.default_rng(seed)
      inbag_idx, oob_idx = self._bootstrap_indices(X.shape[0], rng_b)
      feat_subset = self._choose_features(X, rng_b)

      model = clone(self.base_estimator)

      # Ajuste para evitar o InvalidParameterError
      if hasattr(model, "max_features"):
          if self.task == "classification":
              model.max_features = "sqrt"
          else:
              # Para regressão, calculamos 1/3 das colunas do subconjunto atual
              model.max_features = max(1, int(len(feat_subset) / 3))

      if hasattr(model, "random_state"):
          model.random_state = seed

      model.fit(X.iloc[inbag_idx][feat_subset], y.iloc[inbag_idx])
      return model, oob_idx, feat_subset

    def fit(self, X: pd.DataFrame, y: pd.Series):
        if isinstance(y, pd.DataFrame): y = y.iloc[:, 0]
        rng_master = self._rng()

        if self.task == "classification":
            self.classes_ = np.sort(y.unique())

        seeds = rng_master.integers(0, 1_000_000, size=self.n_estimators)

        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self._train_single_tree)(X, y, seed) for seed in seeds
        )

        self.models_, self.oob_indices_, self.feature_subsets_ = zip(*results)
        self.oob_score_ = self._compute_oob_score(X, y)
        return self

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        if self.task == "regression":
            preds = np.array([m.predict(X[f]) for m, f in zip(self.models_, self.feature_subsets_)])
            return preds.mean(axis=0)

        if self.voting == "soft":
            return self._predict_soft(X)

        preds = np.array([m.predict(X[f]) for m, f in zip(self.models_, self.feature_subsets_)])
        return np.array([pd.Series(preds[:, i]).mode()[0] for i in range(preds.shape[1])])

    def _predict_soft(self, X: pd.DataFrame) -> np.ndarray:
        probas_list = []
        for model, feats in zip(self.models_, self.feature_subsets_):
            p = model.predict_proba(X[feats])

            if not np.array_equal(model.classes_, self.classes_):
                aligned = np.zeros((X.shape[0], len(self.classes_)))
                for idx, cls in enumerate(model.classes_):
                    loc = np.where(self.classes_ == cls)[0][0]
                    aligned[:, loc] = p[:, idx]
                probas_list.append(aligned)
            else:
                probas_list.append(p)

        avg_proba = np.mean(probas_list, axis=0)
        return self.classes_[np.argmax(avg_proba, axis=1)]

    def score(self, X: pd.DataFrame, y: pd.Series) -> float:
        if isinstance(y, pd.DataFrame): y = y.iloc[:, 0]
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred) if self.task == "classification" else root_mean_squared_error(y, y_pred)

    def _compute_oob_score(self, X_train: pd.DataFrame, y_train: pd.Series):
        n = X_train.shape[0]
        oob_preds = [[] for _ in range(n)]

        for model, feats, oob_idx in zip(self.models_, self.feature_subsets_, self.oob_indices_):
            if len(oob_idx) == 0: continue

            preds = model.predict(X_train.iloc[oob_idx][feats])
            for i, p in zip(oob_idx, preds):
                oob_preds[i].append(p)

        y_true, y_pred_final = [], []
        for i in range(n):
            if not oob_preds[i]: continue
            y_true.append(y_train.iloc[i])
            if self.task == "classification":
                y_pred_final.append(pd.Series(oob_preds[i]).mode()[0])
            else:
                y_pred_final.append(np.mean(oob_preds[i]))

        if not y_true: return None
        return accuracy_score(y_true, y_pred_final) if self.task == "classification" else root_mean_squared_error(y_true, y_pred_final)

### Teste 1 - Classificação

In [ ]:
from sklearn.datasets import load_wine

wine = load_wine(as_frame=True)
X_w, y_w = wine.data, wine.target
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_w, y_w, test_size=0.3, stratify=y_w, random_state=42)

rf_fixed = RandomForestManual(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=300,
    task="classification",
    random_state=42
)

rf_fixed.fit(X_train_w, y_train_w)

print("OOB Accuracy:", rf_fixed.oob_score_)
print("Test Accuracy:", rf_fixed.score(X_test_w, y_test_w))

OOB Accuracy: 0.9758064516129032
Test Accuracy: 0.9814814814814815


### Teste 2 - Regressão

In [ ]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True)
X, y = data.frame.drop(columns=['MedHouseVal']), data.frame['MedHouseVal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_fixed = RandomForestManual(
    base_estimator=DecisionTreeRegressor(),
    n_estimators=300,
    task="regression",
    random_state=42
)

rf_fixed.fit(X_train, y_train)

print("OOB RMSE:", rf_fixed.oob_score_)
print("Test RMSE:", rf_fixed.score(X_test, y_test))

OOB RMSE: 0.7921029201780437
Test RMSE: 0.7811466085835139
